In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))
from ast import literal_eval

df = pd.read_csv('../data/processed/pipeline/test/ie_hyperion.csv', converters={'Stralci': literal_eval, 'Repertori': literal_eval})

## Compute predictions with NLTK segmenter

In [2]:
from src.models.nltk_segmenter import NLTKSegmenter

nltk_seg = NLTKSegmenter()
preds = df['Testo'].map(nltk_seg.predict)

[nltk_data] Downloading package punkt to /Users/michele/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Repertories classificaiton

In [ ]:
from src.models.bert_rep import BertRep

cls = BertRep()
output = cls.predict(preds[0])